<a href="https://colab.research.google.com/github/fran-castillo/Proyectos-Python/blob/main/Data_planillas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Scraping de Planillas de Eventos**



## Importamos los modulos necesarios para  la autenticacion  de Google Drive

In [ ]:
from google.colab import drive
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import google.auth

In [ ]:
drive.mount('/content/drive')
auth.authenticate_user()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Utilizamos las Funcion "Listar Carpetas" para listar el directorio completo de carpetas en un carpeta raiz 
*No la utilizamos en el ejercicio pero se usa como ejemplo para otra funcion*

In [ ]:

def listar_carpetas(id_carpeta):
    creds, _ = google.auth.default()
    servicio = build('drive', 'v3', credentials= creds)
    query = f"'{id_carpeta}' in parents and mimeType='application/vnd.google-apps.folder'"
    resultados = servicio.files().list(q=query,fields="nextPageToken, files(id, name)").execute()
    items = resultados.get('files', [])
    for item in items:
        print(f"{item['name']} - {item['id']}")
        query = f"'{item['id']}' in parents and mimeType='application/vnd.google-apps.folder'"
        sub_resultados = servicio.files().list(q=query,fields="nextPageToken, files(id, name)").execute()
        sub_items = sub_resultados.get('files', [])
        for sub_item in sub_items:
            print(f"\t{sub_item['name']} - {sub_item['id']}")
            listar_carpetas(sub_item['id'])

### Utilizamos el ID de la carpeta raiz

In [ ]:
carpeta_raiz_id = 'ID_Carpeta_Raiz'
listar_carpetas(carpeta_raiz_id)

## Definimos la Funcion "Listar Planillas" para obtener el ID todos los archivos Spreadsheet

*Esta funcion devuelve una cantidad erronea de resultados*

In [ ]:
id_list = []
def listar_planillas(id_carpeta):
    creds, _ = google.auth.default()
    servicio = build('drive', 'v3', credentials= creds)
    query = f"'{id_carpeta}' in parents and mimeType='application/vnd.google-apps.folder'"
    resultados = servicio.files().list(q=query,fields="nextPageToken, files(id, name)").execute()
    items = resultados.get('files', [])
    for item in items:
        query = f"'{item['id']}' in parents and mimeType='application/vnd.google-apps.folder'"
        sub_resultados = servicio.files().list(q=query,fields="nextPageToken, files(id, name)").execute()
        sub_items = sub_resultados.get('files', [])
        for sub_item in sub_items:
            listar_planillas(sub_item['id'])
            plani = servicio.files().list(q="name contains 'pax' and mimeType='application/vnd.google-apps.spreadsheet'",fields="nextPageToken, files(id, name)").execute()
            planillas = plani.get('files',[])
            for planilla in planillas:
                  id_list.append(planilla['id'])              
                             

In [ ]:
carpeta_raiz_id = 'ID_Carpeta_Raiz'
listar_planillas(carpeta_raiz_id)

## Por un error en la iteracion el resultado es demasiado alto

In [ ]:
len(id_list)

6200

## Importamos los modulos necesarios para manipular el resultado y obtener la informacion especifica de cada planilla

In [ ]:
import gspread
import numpy as np
from google.auth import default

In [ ]:
creds, _ = default()

## Covertimos el array en una lista para poder buscar resultados repetidos

In [ ]:
id_array= np.array(id_list)

In [ ]:
idList = id_array.tolist()
print(idList)

## Iteramos el array en busca de resultados repetidos
*El resultado desciende de 6200 a 812 pero sigue siendo alto*

In [ ]:
for i in idList:
  if idList.count(i) > 1:
    idList.remove(i)
len(idList)

812

# Definimos la funcion que obtiene la informacion que necesitamos de cada planilla

In [ ]:
filas =[]
def Scrap_planilla(id):
  gc = gspread.authorize(creds)
  sh = gc.open_by_key(id)
  ws = sh.get_worksheet(0)
  fecha = ws.acell('B1').value
  cliente = ws.acell('B3').value
  comercial = ws.acell('B4').value
  pax = ws.acell('B5').value
  ubicacion = ws.acell('B6').value
  tipo = ws.acell('B7').value
  organizador = ws.acell('B8').value
  ambientador = ws.acell('B11').value
  dj = ws.acell('B10').value
  catering = ws.acell('B12').value
  fila = {'Fecha' : fecha,
        'Cliente' : cliente,
        'Comercial' : comercial,
        'Pax' : pax,
        'Ubicacion' : ubicacion,
        'Tipo' : tipo,
        'Organizador' : organizador,
        'Ambientador' : ambientador,
        'DJ' : dj,
        'Catering' : catering,
        }
  filas.append(fila)

In [ ]:
for i in idList[:10]:
  Scrap_planilla(i)

# Importamos pandas para moldear nuestro Dataframe

In [ ]:
import pandas as pd

## Definimos la estructura del Dataframe

In [ ]:
df =pd.DataFrame()
df['Fecha'] = None
df['Cliente'] = None
df['Comercial'] = None
df['Pax'] = None
df['Ubicacion'] = None
df['Tipo'] = None
df['Organizador'] = None
df['Ambientador'] = None
df['DJ'] = None
df['Catering'] = None
df

,Fecha,Cliente,Comercial,Pax,Ubicacion,Tipo,Organizador,Ambientador,DJ,Catering


## Agregamos los datos obtenidos a nuestro Dataframe

In [ ]:
df = df.append(filas, ignore_index= True)
df.head(10)

In [ ]:
df.tail()

# Exportamos el dataframe

In [ ]:
df.to_csv('Data_planillas',encoding='utf-8-sig')